In [24]:
import os
import glob
import pysftp
import datetime as dt
import dateutil as du
import pandas as pd

# First step, retrieving files from a SFTP
The files you want to download are available the 8th of each month.The filename has a date pattern and you just want to download the latest files. Here’s a possible snippet to set an appropriate current_month value :

In [2]:
'''
Generating current_month value to retrieve the appropriate files 
'''
today = dt.date.today()

if today.day >= 8:
    current_month = today.strftime('%Y-%m')
else:
    previous_date = today - du.relativedelta.relativedelta(months=1)
    current_month = previous_date.strftime('%Y-%m')

In [3]:
'''
List of the files to retrieve from the server
'''
file_list = [
    'lego_colors_' + current_month + '.csv',
    'lego_inventory_parts_' + current_month + '.csv',
    'lego_parts_' + current_month + '.csv'
]

In [7]:
'''
SFTP connection variables
'''
local_path  = './sftp_data/'
private_key = '/home/jupyter/.ssh/id_rsa'
hostname    = 'sftp.example.com'
username    = 'sftpuser'
path_sftp   = '/deposit/'

In [ ]:
'''
We retrieve the files with pysftp
'''
cnopts = pysftp.CnOpts()
cnopts.hostkeys = None  

sftp_params = {
    'username': username,
    'private_key' : private_key,
    'cnopts' : cnopts,
}

with pysftp.Connection(hostname, **sftp_params) as sftp:
    with sftp.cd(path_sftp):
        for file in file_list:
            sftp.get(file, localpath=os.path.join(local_path, file), preserve_mtime=True)

In [8]:
'''
Checking if the files are in the correct repository
'''
for local_file in glob.glob(local_path + '*'):
    print(local_file)

./sftp_data/lego_colors_2020-01.csv
./sftp_data/lego_inventory_parts_2020-01.csv
./sftp_data/lego_parts_2020-01.csv


# Now it's time to read the first CSV
We will read the `lego_inventory_parts_2020-01.csv` file and check its structure

In [9]:
inventory = pd.read_csv(os.path.join(local_path, 'lego_inventory_parts_' + current_month + '.csv'))

In [10]:
inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809750 entries, 0 to 809749
Data columns (total 5 columns):
inventory_id    809750 non-null int64
part_num        809750 non-null object
color_id        809750 non-null int64
quantity        809750 non-null int64
is_spare        809750 non-null object
dtypes: int64(3), object(2)
memory usage: 30.9+ MB


In [11]:
inventory.head()

,inventory_id,part_num,color_id,quantity,is_spare
0,1,48379c01,72,1,f
1,1,48395,7,1,f
2,1,mcsport6,25,1,f
3,1,paddle,0,1,f
4,3,11816pr0005,78,1,f


# Joining the other CSV files to make it human readable
We will join the `color` CSV file and join on the `color_id`, then we will join the `parts` CSV file and join it on `part_num`

In [12]:
colors = pd.read_csv(os.path.join(local_path, 'lego_colors_' + current_month + '.csv'))

In [13]:
colors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 4 columns):
id          182 non-null int64
name        182 non-null object
rgb         182 non-null object
is_trans    182 non-null object
dtypes: int64(1), object(3)
memory usage: 5.8+ KB


In [14]:
inventory = inventory.merge(colors, how='inner', left_on='color_id', right_on='id')

In [15]:
parts = pd.read_csv(os.path.join(local_path, 'lego_parts_' + current_month + '.csv'))

In [17]:
parts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35104 entries, 0 to 35103
Data columns (total 4 columns):
part_num            35104 non-null object
name                35104 non-null object
part_cat_id         35104 non-null int64
part_material_id    35104 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.1+ MB


In [16]:
inventory = inventory.merge(parts, how='inner', left_on='part_num', right_on='part_num', suffixes=('_color', '_part'))

In [18]:
inventory.head()

,inventory_id,part_num,color_id,quantity,is_spare,id,name_color,rgb,is_trans,name_part,part_cat_id,part_material_id
0,1,48379c01,72,1,f,72,Dark Bluish Gray,6C6E68,f,Sports Promo Figure Base with Feet,27,1
1,15767,48379c01,72,1,f,72,Dark Bluish Gray,6C6E68,f,Sports Promo Figure Base with Feet,27,1
2,738,48379c01,0,1,f,0,Black,05131D,f,Sports Promo Figure Base with Feet,27,1
3,11896,48379c01,15,1,f,15,White,FFFFFF,f,Sports Promo Figure Base with Feet,27,1
4,1733,48379c01,1,1,f,1,Blue,0055BF,f,Sports Promo Figure Base with Feet,27,1


# Dropping columns we won't use
We are cleaning our dataframe from redundant information and some we won't need to import in our database

In [20]:
inventory = inventory.drop(['color_id', 'id', 'rgb', 'part_cat_id', 'part_material_id'], axis=1)

In [21]:
inventory.head()

,inventory_id,part_num,quantity,is_spare,name_color,is_trans,name_part
0,1,48379c01,1,f,Dark Bluish Gray,f,Sports Promo Figure Base with Feet
1,15767,48379c01,1,f,Dark Bluish Gray,f,Sports Promo Figure Base with Feet
2,738,48379c01,1,f,Black,f,Sports Promo Figure Base with Feet
3,11896,48379c01,1,f,White,f,Sports Promo Figure Base with Feet
4,1733,48379c01,1,f,Blue,f,Sports Promo Figure Base with Feet


# Our dataframe is ready to be saved !
We will save this dataframe to a CSV file. And, we will archive the files by moving them in an `archive` repository.

In [25]:
csv_options = {
    'path_or_buf': os.path.join(local_path, 'inventory_enriched.csv'),
    'sep'      : ';',
    'encoding' : 'utf8',
    'index'    : False,
    'na_rep'   : '\\N'
}

In [26]:
inventory.to_csv(**csv_options)

In [28]:
!head './sftp_data/inventory_enriched.csv'

inventory_id;part_num;quantity;is_spare;name_color;is_trans;name_part
1;48379c01;1;f;Dark Bluish Gray;f;Sports Promo Figure Base with Feet
15767;48379c01;1;f;Dark Bluish Gray;f;Sports Promo Figure Base with Feet
738;48379c01;1;f;Black;f;Sports Promo Figure Base with Feet
11896;48379c01;1;f;White;f;Sports Promo Figure Base with Feet
1733;48379c01;1;f;Blue;f;Sports Promo Figure Base with Feet
2304;48379c01;1;f;Blue;f;Sports Promo Figure Base with Feet
7378;48379c01;1;f;Tan;f;Sports Promo Figure Base with Feet
3157;48379c01;1;f;Dark Blue;f;Sports Promo Figure Base with Feet
22;15573;1;f;Dark Bluish Gray;f;Plate Special 1 x 2 with 1 Stud with Groove and Inside Stud Holder (Jumper)


In [29]:
os.rename(os.path.join(local_path, 'lego_inventory_parts_' + current_month + '.csv'), os.path.join(local_path, 'archive', 'lego_inventory_parts_' + current_month + '.csv'))
os.rename(os.path.join(local_path, 'lego_colors_' + current_month + '.csv'), os.path.join(local_path, 'archive', 'lego_colors_' + current_month + '.csv'))
os.rename(os.path.join(local_path, 'lego_parts_' + current_month + '.csv'), os.path.join(local_path, 'archive', 'lego_parts_' + current_month + '.csv'))

# Now we upload it on our Google Cloud Storage bucket

In [ ]:
!gsutil mv './sftp_data/inventory_enriched.csv' gs://my-gcs-bucket/